# OneStopEnglish: Exploratory Data Analysis

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import spacy
import re
import textstat
import os
from os import listdir
from os.path import join, isfile
import chardet

In [ ]:
# Sets option to display all text in pandas dataframes
pd.set_option('display.max_colwidth', None)

In [ ]:
# Get file path for OneStopEnglish Corpus from individual TXT files.
FilePath = os.getcwd() + '/OneStopEng/TextByReadingLevel/'

# For some reason there are two "Int" files--do not import the first, as there 
SubDir = [join(FilePath, f)+'/' for f in listdir(FilePath) if f != 'Int-Txt']

# Load all texts into pandas data frame
TextDF = pd.DataFrame()

for s in SubDir:
    TextFiles = listdir(s)

    for t in TextFiles:
        # There is a hidden .ds_store file that should be skipped when importing data
        if t == '.DS_Store':
            pass
        else:
            file = open(s + t, 'r')
            text = file.read()
            file.close()

            # Append text data and file name to dataframe
            TextDF = TextDF.append({'file_nm' : t,
                                    'text' : text},
                                   ignore_index = True)

In [ ]:
# Isolate name and level of text and drop file_nm
TextDF['name'] = TextDF['file_nm'].str[:-8]
TextDF['level'] = TextDF['file_nm'].str[-7:-4]
TextDF['text'] = TextDF['text'].apply(str)

TextDF.groupby(['level'])['text'].count()

# Intermediate text is labeled--this should be removed
# TextDF['text'] = TextDF['text'].map(lambda x: x.lstrip('Intermediate'))
# TextDF['text'] = TextDF['text'].str.replace('\n', ' ')

In [ ]:
# Compute text difficulty using textstat
TextDF['difficulty'] = TextDF['text'].apply(textstat.flesch_reading_ease)

# Compute different statistics for text entries
TextDF['n_sent'] = TextDF['text'].apply(textstat.sentence_count)
TextDF['n_syll'] = TextDF['text'].apply(textstat.syllable_count)
TextDF['n_lex'] = TextDF['text'].apply(textstat.lexicon_count)
TextDF['syll_sent'] = TextDF['n_syll']/TextDF['n_sent']

In [ ]:
TextDF.groupby('level')['difficulty'].plot(kind = 'hist', legend = True, alpha = 0.5)

# Basic ML Models

In [ ]:
# Set seed so results can be reproduced
randomseed = 20200922

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# Preprocess data

# Label encoder
le = preprocessing.LabelEncoder()

# Define predictor variables
PredVars = ['difficulty', 'n_lex', 'n_syll', 'n_sent']

# Subset TextDF to desired variables
MLDat = TextDF[['level']+PredVars].reset_index().drop(columns = ['index'])
MLDat['level'] = le.fit_transform(MLDat['level'])

# Target and predictor variables
y = MLDat['level']
X = MLDat[PredVars]

# Split data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = randomseed)

In [ ]:
# Gaussian Naive Bayes Model
gnb = GaussianNB()
pred = gnb.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, pred, normalize = True)

In [ ]:
# Linear support verctor classification
svc = LinearSVC(random_state = randomseed)
pred = svc.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, pred, normalize = True)

In [ ]:
# K neighbors classifier
knc = KNeighborsClassifier(n_neighbors = 3)
knc.fit(X_train, y_train)
pred = knc.predict(X_test)
accuracy_score(y_test, pred, normalize = True)

In [ ]:
# Multinomial logistic regression
mlr = LogisticRegression(max_iter = 450)
pred = mlr.fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, pred, normalize = True)

In [ ]:
###################################
# IMPORT DATA STORED IN CSV FILES #
###################################

# Get file path to OneStopEnglish Corpus including combined CSV files.
FilePath = os.getcwd() + '/OneStopEng/TextOneCSVPerFile/'

# Texts are organized by difficulty level
Files = listdir(FilePath)

# Import all texts into pandas dataframe
oseDF = pd.DataFrame() # Initialize dataframe

# Loop through all files
for f in Files:
    
    # Files have different encodings
    # Find the encoding for each file and use that in read_csv command
    with open(FilePath + f, 'rb') as rawdat:
        result = chardet.detect(rawdat.read(10000))
        
    oseDFA = pd.read_csv(FilePath + f, encoding = result['encoding'])
    oseDFA['name'] = f
    
    oseDF = oseDF.append(oseDFA)